In [1]:
!pip install -I xpander-sdk openai python-dotenv

  Using cached xpander_sdk-1.19.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached jsii-1.106.0-py3-none-any.whl.metadata (79 kB)
  Using cached publication-0.0.3-py2.py3-none-any.whl.metadata (8.7 kB)
  Using cached typeguard-4.2.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.10.3-py3-none-any.whl.metadata (172 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using

In [3]:
from xpander_sdk import XpanderClient, LLMProvider
from openai import OpenAI
from dotenv import load_dotenv
import os

## Load environment variables
load_dotenv()
friendli_client = OpenAI(
    base_url="https://api.friendli.ai/serverless/v1",
    api_key=os.environ.get("FRIENDLI_TOKEN")
)

xpanderAPIKey = os.environ.get("XPANDER_API_KEY","")
xpanderAgentID = os.environ.get("XPANDER_AGENT_ID", "")


xpander_client = XpanderClient(api_key=xpanderAPIKey)
xpander_agent = xpander_client.agents.get(agent_id=xpanderAgentID)

print(xpander_agent.get_tools())


[{'type': 'function', 'function': {'name': 'PgLinkedInCompanyGetInfoByDomain', 'description': "This operation retrieves detailed information about a company from LinkedIn using its domain. It's useful for professionals looking to gather insights on potential partners, competitors, or job opportunities. This operation is particularly beneficial for recruiters, sales teams, and market researchers who need quick access to company profiles and related data.", 'parameters': {'type': 'object', 'properties': {'queryParams': {'type': 'object', 'properties': {}, 'required': []}, 'pathParams': {'type': 'object', 'properties': {}, 'required': []}, 'bodyParams': {'type': 'object', 'properties': {'inputTask': {'type': 'string', 'description': 'input user sub task based on the input query'}}, 'required': []}}, 'required': ['query_params', 'path_params', 'body_params']}}}, {'type': 'function', 'function': {'name': 'PgStockDataFetchAndEmailSummary', 'description': 'This operation fetches stock data an

In [9]:
# verify the friendli client is working
completion = friendli_client.chat.completions.create(
    model="meta-llama-3.1-8b-instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Hello. It's nice to meet you. Is there anything I can help you with or would you like to chat?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [5]:
print(xpander_agent.get_tools())

[{'type': 'function', 'function': {'name': 'PgLinkedInCompanyGetInfoByDomain', 'description': "This operation retrieves detailed information about a company from LinkedIn using its domain. It's useful for professionals looking to gather insights on potential partners, competitors, or job opportunities. This operation is particularly beneficial for recruiters, sales teams, and market researchers who need quick access to company profiles and related data.", 'parameters': {'type': 'object', 'properties': {'queryParams': {'type': 'object', 'properties': {}, 'required': []}, 'pathParams': {'type': 'object', 'properties': {}, 'required': []}, 'bodyParams': {'type': 'object', 'properties': {'inputTask': {'type': 'string', 'description': 'input user sub task based on the input query'}}, 'required': []}}, 'required': ['query_params', 'path_params', 'body_params']}}}, {'type': 'function', 'function': {'name': 'PgStockDataFetchAndEmailSummary', 'description': 'This operation fetches stock data an

In [10]:
memory = [{"role": "system", "content": "You are a helpful assistant with function calling and tool access. you are running in While loop if you want to stop the loop please add ##FINAL ANSWER## in your answer. You are using xpander AI agent graph, when selecting tool you will get access to antoher tool only after execution. You need to execute the tool in order to get access to more tools, the xpander.ai might respond with system messages about selected group that indicated that you will get access in the next attemp."},
          {"role": "user", "content": "Find xpander.ai Linkedin Page"}
          ]
print(memory[-1])

number_of_calls = 1
while True:
    llm_response = friendli_client.chat.completions.create(
        model="meta-llama-3.1-8b-instruct",
        messages=memory,
        tools=xpander_agent.get_tools(),
        tool_choice="auto"
    )
    memory.append({"role": "assistant", "content": f'Step number: {number_of_calls}'})
    print(memory[-1])

    memory.append(llm_response.choices[0].message)
    print(memory[-1])
    model_response = llm_response.choices[0].message
    if model_response.tool_calls:
        print(model_response.tool_calls)
        tools_to_run = XpanderClient.extract_tool_calls(llm_response=llm_response.model_dump())
        print("Tool Name:", tools_to_run[0].name)
        tool_responses = xpander_agent.run_tools(tool_calls=tools_to_run)
        for tool_response in tool_responses:
            memory.append({"role": "tool", "content": tool_response.result, "tool_call_id": tool_response.tool_call_id})
            print(memory[-1])
    if (model_response.content):
        if "##FINAL ANSWER##" in model_response.content:
            break
    number_of_calls += 1
print(model_response.content)   

{'role': 'user', 'content': 'Find xpander.ai Linkedin Page'}
{'role': 'assistant', 'content': 'Step number: 1'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_12hRxcFk7Udlw3n0SLV5fLcJ', function=Function(arguments='{"queryParams": {}, "pathParams": {}, "bodyParams": {"inputTask": "Find xpander.ai Linkedeon page"}}', name='PgLinkedInCompanyGetInfoByDomain'), type='function')])
[ChatCompletionMessageToolCall(id='call_12hRxcFk7Udlw3n0SLV5fLcJ', function=Function(arguments='{"queryParams": {}, "pathParams": {}, "bodyParams": {"inputTask": "Find xpander.ai Linkedeon page"}}', name='PgLinkedInCompanyGetInfoByDomain'), type='function')]
Tool Name: PgLinkedInCompanyGetInfoByDomain
{'role': 'tool', 'content': "system message: graph prompt group selected, ignore this and proceed with the user's request using new tools.", 'tool_call_id': 'call_12hRxcFk7Udlw3n0SLV5fLcJ'}
{'role': 'assistant', 'c